In [18]:
import numpy as np
from math import *

def init_const():
    global x_0, y_0, form_x, form_y, d, N
    
    x_0, y_0 = -2, 2
    form_x = 'x * x - y * y'
    form_y = '2 * x * y - 0.2'
    d = 0.0125
    N = int(4 / d)

def formula(x, y):
    return(eval(form_x), eval(form_y))

def get_num(x, y):
    y = np.where(x < -2, 3, y)
    x = np.where(x < -2, -3, x)
    
    y = np.where(x > 2, 3, y)
    x = np.where(x > 2, -3, x)
    
    x = np.where(y < -2, -3, x)
    y = np.where(y < -2, 3, y)
    
    x = np.where(y > 2, -3, x)
    y = np.where(y > 2, 3, y)
    
    i = ((y_0 - y) // d * N) + ((x - x_0) // d) + 1
    i = np.where(i < 0, 0, i)
    return i[np.where(i > 0)]

def new_num(old_num):
    return 4 * N * ((old_num - 1) // N) + 2 * ((old_num - 1) % N) + 1

def make_graph(graph_keys=None, save=False):
    global x_0, y_0, d, N, form_x, form_y, graph
    if graph_keys == None:
        init_const()
        graph = dict.fromkeys(range(1, N * N + 1))
    else:
#         graph_keys = np.array(scc[np.argsort([len(comp) for comp in scc])[-1]])
        graph_verts = []
        dots = new_num(np.array(graph_keys))
        N = N * 2
        for dot in dots:
            graph_verts += [dot, dot + 1, dot + N, dot + N + 1]
        new_graph = dict.fromkeys(graph_verts)
        graph = new_graph
        d = d / 2
        
    pattern_x = np.ones((10, 10)) * np.array(range(1, 11)) * 0.1 * d - d / 20
    pattern_y = pattern_x.copy().transpose()
    
    e_cntr = 0
    
    for n in graph.keys():
        y_n = round(y_0 - ((n - 1) // N) * d, 4)
        x_n = round(x_0 + ((n - 1) % N) * d, 4)
        ss = np.unique(get_num(*formula(x_n + pattern_x, y_n - pattern_y)))
        ss = np.array([sss for sss in ss if sss in graph.keys()])
        s = set(ss.astype('int32'))
        graph[n] = s
        e_cntr += len(s)
            
    if save:
        file = 'graph_' + str(N*N) + '.txt'
        save_graph(file, graph)
    return graph
    
def save_graph(file, graph):
    f = open(file, 'w')
    for k, v in graph.items():
        f.write(f'{k}: {v}\n')
    f.close()


In [46]:
def invert_graph(graph):
    i_graph = dict.fromkeys(graph.keys(), 0)
    for k, v in graph.items():
        for vertice in v:
            if i_graph[vertice]:
                i_graph[vertice].add(int(k))
            else:
                i_graph[vertice] = {int(k)}
    for k, v in i_graph.items():
        if v == 0:
            i_graph[k] = set()
    return i_graph

def dfs(graph, v, comp, visited):
    visited[v] = True
    end = True
    for e in graph[v]:
        if not visited[e]:
            end = False
            dfs(graph, e, comp, visited)
    comp.append(v)

def kosaraju(graph):
    i_graph = invert_graph(graph)
    n = N * N
    
    visited = [False] * (n + 1)
    coms = []
    
    for i in range(1, n + 1):
        if (i in graph.keys() and (not visited[i])):
#         if not visited[i]:
            comp = []
            dfs(graph, i, comp, visited)
            coms.append(comp)

    visited = [False] * (n + 1)
    c = []

    for comp in coms[::-1]:
        for v in comp[::-1]:
            if not visited[v]:
                co = []
                dfs(i_graph, v, co, visited)
                if len(co) > 0:
                    c.append(co)
                    
    return c

In [35]:
from PIL import Image
from PIL import ImageDraw
from PIL import ImageColor

def scale(x, y):
    return (x + 2) * width // 4 , height - (y + 2) * height // 4

def fill_square(number, d, color):
    y, x = divmod(number - 1, N)
    x, y = x_0 + x * d, y_0 - y * d
    draw.rectangle((*scale(x, y), *scale(x + d, y - d)), fill=color)
    
def draw_grid():
    x, y = 0, 0
    if d > 0.01:
        while x < width:
            x += width / N
            draw.line((x, 0, x, height), fill=ImageColor.getrgb("blue"))
        while y < height:
            y += height / N
            draw.line((0, x, width, x), fill=ImageColor.getrgb("blue"))
            
def draw_axis():
    zero_x = width / 2
    zero_y = height / 2

    draw.line((0, zero_x + 1, height, zero_x + 1))
    draw.line((zero_y + 1, 0, zero_y + 1, width))

def draw_scc(scc, grid=False, axis=False, save=True):
    global height, width, draw
    height, width = 800, 800
    image = Image.new("RGB", (width, height))
    draw = ImageDraw.Draw(image)
    draw.rectangle((0, 0, width, height), fill='white')
    k = 0
    for comp in scc:
        if len(comp) > 1:
            print('drawing component')
            for v in comp:
                fill_square(v, d, 'blue')

    if axis:
        draw_axis()

    if grid:
        draw_grid()

    if save:
        im_name = 'test_image_' + str(N * N) + '.png'
        image.save(im_name, "PNG")


In [4]:
%%time
graph = make_graph()
scc = kosaraju(graph)
draw_scc(scc)
# n = 3
# for _ in range(n):
#     graph_keys = scc[np.argsort([len(comp) for comp in scc])[-1]]
#     graph = make_graph(graph_keys)
#     scc = kosaraju(graph)
#     draw_scc(scc)


Wall time: 6.22 s


In [21]:
def old_num(new_num):
    return (((new_num - 1) // N) // 2) * N // 2 + ((new_num - 1) % N) // 2 + 1

In [29]:
def make2_graph(graph_keys=None, save=False):
    global x_0, y_0, d, N, form_x, form_y, graph
    if graph_keys == None:
        init_const()
        graph = dict.fromkeys(range(1, N * N + 1))
    else:
        graph_verts = []
        dots = np.unique(old_num(np.array(graph_keys)))
        N = int(N / 2)
#         for dot in dots:
#             graph_verts += [dot, dot + 1, dot + N, dot + N + 1]
        new_graph = dict.fromkeys(dots)
        graph = new_graph
        d = d * 2
        
    pattern_x = np.ones((10, 10)) * np.array(range(1, 11)) * 0.1 * d - d / 20
    pattern_y = pattern_x.copy().transpose()
    
    e_cntr = 0
    
    for n in graph.keys():
        y_n = round(y_0 - ((n - 1) // N) * d, 4)
        x_n = round(x_0 + ((n - 1) % N) * d, 4)
        ss = np.unique(get_num(*formula(x_n + pattern_x, y_n - pattern_y)))
        ss = np.array([sss for sss in ss if sss in graph.keys()])
        s = set(ss.astype('int32'))
        graph[n] = s
        e_cntr += len(s)
            
    if save:
        file = 'graph_' + str(N*N) + '.txt'
        save_graph(file, graph)
    return graph

In [63]:
graph = make_graph()
scc = kosaraju(graph)
draw_scc(scc)

drawing component
drawing component


In [64]:
graph_keys = []
for comp in scc:
    if len(comp) > 1:
        graph_keys += comp

In [65]:
graph2 = make2_graph(graph_keys)
scc2 = kosaraju(graph2)
# print(scc2)
draw2_scc(scc2)

drawing component
drawing component
drawing component
drawing component
drawing component
drawing component


In [59]:
draw2_scc(scc2)

drawing component
drawing component
drawing component
drawing component
drawing component
drawing component


In [17]:
N = 6
new_num = 13
old_num(new_num)

4

In [58]:
def draw2_scc(scc, grid=False, axis=False, save=True):
    global height, width, draw
    height, width = 800, 800
    image = Image.new("RGB", (width, height))
    draw = ImageDraw.Draw(image)
    draw.rectangle((0, 0, width, height), fill='white')
    k = 0
    colors = ['blue', 'green', 'orange', 'red', 'purple', 'black']
    i = 0
    for comp in scc:
        print('drawing component')
        for v in comp:
            fill_square(v, d, colors[i])
        i += 1
            
    
    if axis:
        draw_axis()

    if grid:
        draw_grid()

    if save:
        im_name = 'test_image_' + str(N * N) + '.png'
        image.save(im_name, "PNG")


In [45]:
len(scc)

2

In [69]:
graph_keys = []
for comp in scc2:
    graph_keys += comp
        
graph2 = make2_graph(graph_keys)
scc2 = kosaraju(graph2)
draw2_scc(scc2)

drawing component
drawing component
